<a href="https://colab.research.google.com/github/NilakshanRaveendran/Text-Summarizer-Transformer-Model/blob/main/TextSummarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers accelerate datasets

In [ ]:
from datasets import load_dataset

# The "multi_news" dataset relies on a deprecated method of loading via a Python script.
# We're replacing it with "cnn_dailymail", a well-supported summarization dataset.
multinews = load_dataset("cnn_dailymail", "3.0.0", split = 'test')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
multinews.to_pandas()

,article,highlights,id
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,f001ec5c4704938247d27a44948eebb37ae98d01
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,4495ba8f3a340d97a9df1476f8a35502bcce1f69
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f
...,...,...,...
11485,Telecom watchdogs are to stop a rip-off that a...,Operators are charging up to 20p a minute - ev...,0ac776a4dc09ca97c136f4314fed4defb48a361a
11486,The chilling reenactment of how executions are...,Bali Nine ringleaders will face the firing squ...,fe89a6a2e28d173e5ad4c6d814c15b95aa969e3f
11487,It is a week which has seen him in deep water ...,Hardy was convicted of domestic abuse against ...,ded2f535cd6ab95d11b5f4ea29bbf2b2d3c55c50
11488,"Despite the hype surrounding its first watch, ...",Apple sold more than 61 million iPhones in the...,30ec5f280eee772a73d181bfc8514defd8026434


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
multi_news = multinews.train_test_split(test_size=0.2)

In [ ]:
prefix = "Summarizer"

def process_function(examples):
  input = [prefix + doc for doc in examples["article"]]
  model_inputs = tokenizer(input, max_length= 1024, truncation= True)
  labels = tokenizer(text= examples["highlights"], max_length= 128, truncation= True)
  model_inputs["labels"] = labels["input_ids"]


  return model_inputs

In [ ]:
tokenized_multi_news = multi_news.map(process_function, batched= True)

Map:   0%|          | 0/9192 [00:00<?, ? examples/s]

Map:   0%|          | 0/2298 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model= model )

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
trainings_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    fp16=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=trainings_args,
    train_dataset=tokenized_multi_news["train"],
    eval_dataset=tokenized_multi_news["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-3263220153.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nilaksh2001 (nilaksh2001-nix) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,2.041100,1.770393
2,1.902600,1.748830
3,1.878800,1.736974
4,1.861800,1.729822
5,1.853400,1.727054
6,1.832200,1.722364
7,1.828300,1.720985
8,1.818900,1.719783
9,1.821300,1.718905
10,1.819200,1.718280


TrainOutput(global_step=9200, training_loss=1.8639542753800102, metrics={'train_runtime': 6359.8074, 'train_samples_per_second': 14.453, 'train_steps_per_second': 1.447, 'total_flos': 2.4865657672433664e+16, 'train_loss': 1.8639542753800102, 'epoch': 10.0})

In [ ]:
text = """
This article covers everything you need to learn about AI, ML and Data Science, starting with Python programming, statistics and probability. It also includes EDA, visualization, ML, deep learning, AI, projects and interview questions for career preparation.

1. Learning Python
Python is one of the most popular programming languages today, known for its simplicity, extensive features and library support. Its clean syntax makes it beginner-friendly, while its libraries and frameworks makes it perfect for developers.


2. Math For Data Science
Math for Data Science is all about the fundamental mathematical tools and concepts you need to work effectively with data. It includes Statistics & Probability, Linear Algebra and Calculus.


3. Exploratory Data Analysis
Exploratory Data Analysis (EDA) is an approach to analyzing data sets to summarize their main characteristics, often using visual methods. It involves understanding data, cleaning data, visualizing data and further analysis.


4. Data Analysis
Data Analysis is the technique of collecting, transforming and organizing data to make future predictions and informed data-driven decisions. It also helps to find possible solutions for a business problem.
There are six steps for Data Analysis which are: Ask or Specify Data Requirements, Prepare or Collect Data, Clean and Process, Analyze, Share, Act or Report.


5. Data Visualization
Data visualization is the process of turning data into visual representations like charts, graphs and maps. It helps us understand trends, patterns and outliers.


6. Machine Learning
Machine learning is a subset of Artificial Intelligence (AI) that enables computers to learn from data and make predictions without being explicitly programmed.

It can be categorized into three types: Supervised Learning, Unsupervised Learning and Reinforcement Learning.


7. Data Science with Python
Data science enables organizations to make informed decisions, solve problems and understand human behavior. As the volume of data grows, so does the demand for skilled data scientists. The most common languages used for data science are Python and R, with Python being particularly popular.


8. Deep Learning
Deep Learning is a branch of Artificial Intelligence (AI) that enables machines to learn from large amounts of data. It uses neural networks with many layers to automatically find patterns and make predictions.

9. Artificial Intelligence
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and act like humans.


10. Generative AI & LLM
Generative AI (Gen AI) is a branch of artificial intelligence that can create new content instead of just analyzing data. It uses machine learning models (like large language models, GANs, and diffusion models) to generate text, images, audio, code, or even video.

LLM (Large Language Model) is a type of artificial intelligence model designed to understand and generate human-like language.
"""

In [ ]:
input_ids = tokenizer(text, return_tensors="pt", max_length= 1024, truncation= True).input_ids
input_ids = input_ids.to('cuda')

In [ ]:
import torch

with torch.no_grad():
  if model.device.type == 'cuda':
    output = model.generate(input_ids, max_length = 128, num_beams = 5)


summary_ids = output[0].tolist()

summary = tokenizer.decode(summary_ids, skip_special_tokens=True)
print(summary)

Data Analysis Data Analysis is the technique of collecting, transforming and organizing data to make future predictions and informed data-driven decisions. Data Visualization Data visualization is the process of turning data into visual representations like charts, graphs and maps. It helps us understand trends, patterns and outliers.


In [ ]:
ref_summary = """
This article provides a comprehensive overview of AI, ML, and Data Science, focusing on key areas such as Python programming, statistics, exploratory data analysis (EDA), data visualization, and machine learning. It emphasizes the importance of Python for its user-friendly nature and extensive libraries. Math fundamentals like statistics, linear algebra, and calculus are essential for data science. EDA involves analyzing datasets visually to summarize their characteristics. Data analysis comprises steps like data collection and reporting, while data visualization helps uncover trends through graphical representations. Machine learning, as a subset of AI, allows computers to make predictions based on data. The article also explains deep learning, generative AI, and large language models, highlighting their applications in producing new content and simulating human intelligence.
"""

In [ ]:
summary

'Data Analysis Data Analysis is the technique of collecting, transforming and organizing data to make future predictions and informed data-driven decisions. Data Visualization Data visualization is the process of turning data into visual representations like charts, graphs and maps. It helps us understand trends, patterns and outliers.'

In [ ]:
!pip install rouge_score
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(ref_summary, summary)
scores

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=91de8f6451b312659424e643d7749d779605d3d7667a5cf4fa992d23df1d6898
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


{'rouge1': Score(precision=0.6041666666666666, recall=0.23770491803278687, fmeasure=0.34117647058823525),
 'rouge2': Score(precision=0.10638297872340426, recall=0.04132231404958678, fmeasure=0.059523809523809514),
 'rougeL': Score(precision=0.3333333333333333, recall=0.13114754098360656, fmeasure=0.18823529411764708)}

In [ ]:
trainer.save_model()

In [ ]:
model.save_pretrained('Summarizer')